# Generating a Forced ILP Dataset

Finds a group of small buffer overflow examples from the Juliet dataset, then generates their Prolog representations from the code property graph generated by Joern. These are saved to `../data/ilp_data.csv.gz`  and `../data/ilp_prolog_data.csv.gz` respectively. 


## Find a nice subset

Find a small number of short buffer overflow examples from the Juliet dataset, and save them into the `../data/ilp_data_force.csv.gz` dataframe. 

In [35]:
import pandas as pd

In [36]:
buffer_overflow_juliet = pd.read_csv("../data/buffer_overflow_data.csv.gz")

In [37]:
buffer_overflow_juliet['code_length'] = buffer_overflow_juliet.code.apply(len)
buffer_overflow_juliet

,Unnamed: 0,Unnamed: 0.1,testcase_ID,filename,code,flaw,flaw_loc,bug,code_length
0,984,984,62516,000/062/516/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,112,False,5108
1,985,985,62517,000/062/517/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,117,False,9668
2,986,986,62518,000/062/518/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,117,False,9710
3,987,987,62519,000/062/519/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,123,False,10162
4,988,988,62520,000/062/520/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,123,False,10084
5,989,989,62521,000/062/521/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,122,False,10123
6,990,990,62522,000/062/522/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,122,False,10054
7,991,991,62523,000/062/523/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,130,False,10190
8,992,992,62524,000/062/524/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,117,False,9897
9,993,993,62525,000/062/525/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,117,False,9799


In [38]:
buffer_overflow_juliet.groupby(['testcase_ID']).apply(
    lambda testcase, **kwargs: sum(testcase['code_length']),
    axis='columns',
).sort_values()

testcase_ID
-62852      1570
-62900      1576
-62869      1616
-62804      1618
-232012     1619
-62853      1621
-62917      1622
-232086     1625
-62901      1627
-62854      1630
-62867      1632
-62902      1636
-62915      1638
-62868      1643
-62861      1649
-62916      1649
-62909      1655
-62865      1657
-62913      1663
-62821      1664
-232029     1665
-231979     1667
-62805      1669
-62860      1670
-232103     1671
 62852      1674
-232013     1674
-62908      1676
-62862      1676
-62806      1678
           ...  
 232342    11388
 67724     11389
 232346    11425
 232341    11486
 232347    11501
 232196    11511
 67718     11512
 232343    11514
 232345    11523
 67716     11542
 70687     11546
 67717     11581
 67759     11584
 67715     11620
 232339    11643
 67719     11648
 232337    11673
 232364    11690
 62727     11694
 232338    11712
 232336    11751
 232340    11779
 232184    11843
 232352    12397
 67552     12449
 67744     12616
 62548     13047
 6

Pick 20 smallest:

In [39]:
ilp_data= buffer_overflow_juliet[
    (buffer_overflow_juliet['testcase_ID'] == 62852) | 
    (buffer_overflow_juliet['testcase_ID'] == 62900) | 
    (buffer_overflow_juliet['testcase_ID'] == 62869) | 
    (buffer_overflow_juliet['testcase_ID'] == 62804) | 
    (buffer_overflow_juliet['testcase_ID'] == 232012) | 
    (buffer_overflow_juliet['testcase_ID'] == 62853) |
    (buffer_overflow_juliet['testcase_ID'] == 62917) |
    (buffer_overflow_juliet['testcase_ID'] == 232086) |
    (buffer_overflow_juliet['testcase_ID'] == 62901) |
    (buffer_overflow_juliet['testcase_ID'] == 62854) |
    (buffer_overflow_juliet['testcase_ID'] == 62867) |
    (buffer_overflow_juliet['testcase_ID'] == 62902) |
    (buffer_overflow_juliet['testcase_ID'] == 62915) |
    (buffer_overflow_juliet['testcase_ID'] == 62868) |
    (buffer_overflow_juliet['testcase_ID'] == 62861) |
    (buffer_overflow_juliet['testcase_ID'] == 62916) |
    (buffer_overflow_juliet['testcase_ID'] == 62909) |
    (buffer_overflow_juliet['testcase_ID'] == 62865) |
    (buffer_overflow_juliet['testcase_ID'] == 62913) |
    (buffer_overflow_juliet['testcase_ID'] == 62821) |
    (buffer_overflow_juliet['testcase_ID'] == -62852) | 
    (buffer_overflow_juliet['testcase_ID'] == -62900) | 
    (buffer_overflow_juliet['testcase_ID'] == -62869) | 
    (buffer_overflow_juliet['testcase_ID'] == -62804) | 
    (buffer_overflow_juliet['testcase_ID'] == -232012) |
    (buffer_overflow_juliet['testcase_ID'] == -62853) |
    (buffer_overflow_juliet['testcase_ID'] == -62917) |
    (buffer_overflow_juliet['testcase_ID'] == -232086) |
    (buffer_overflow_juliet['testcase_ID'] == -62901) |
    (buffer_overflow_juliet['testcase_ID'] == -62854) |
    (buffer_overflow_juliet['testcase_ID'] == -62867) |
    (buffer_overflow_juliet['testcase_ID'] == -62902) |
    (buffer_overflow_juliet['testcase_ID'] == -62915) |
    (buffer_overflow_juliet['testcase_ID'] == -62868) |
    (buffer_overflow_juliet['testcase_ID'] == -62861) |
    (buffer_overflow_juliet['testcase_ID'] == -62916) |
    (buffer_overflow_juliet['testcase_ID'] == -62909) |
    (buffer_overflow_juliet['testcase_ID'] == -62865) |
    (buffer_overflow_juliet['testcase_ID'] == -62913) |
    (buffer_overflow_juliet['testcase_ID'] == -62821)
]
ilp_data

,Unnamed: 0,Unnamed: 0.1,testcase_ID,filename,code,flaw,flaw_loc,bug,code_length
516,1500,1500,62804,000/062/804/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,1722
533,1517,1517,62821,000/062/821/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,35,False,1808
581,1565,1565,62852,000/062/852/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,30,False,1674
582,1566,1566,62853,000/062/853/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2396
583,1567,1567,62854,000/062/854/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2414
590,1574,1574,62861,000/062/861/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2453
594,1578,1578,62865,000/062/865/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2468
596,1580,1580,62867,000/062/867/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,34,False,1781
597,1581,1581,62868,000/062/868/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,34,False,1790
598,1582,1582,62869,000/062/869/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,32,False,1760


In [40]:
ilp_data = ilp_data.drop(["Unnamed: 0", "Unnamed: 0.1"], axis='columns')

In [41]:
ilp_data

,testcase_ID,filename,code,flaw,flaw_loc,bug,code_length
516,62804,000/062/804/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,1722
533,62821,000/062/821/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,35,False,1808
581,62852,000/062/852/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,30,False,1674
582,62853,000/062/853/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2396
583,62854,000/062/854/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2414
590,62861,000/062/861/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2453
594,62865,000/062/865/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,2468
596,62867,000/062/867/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,34,False,1781
597,62868,000/062/868/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,34,False,1790
598,62869,000/062/869/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,32,False,1760


Looking at scripts that we can generate a forced example from

Below is what we would like our faked code to look like:

In [42]:
good_fake = """
/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01.c
Label Definition File: CWE121_Stack_Based_Buffer_Overflow__CWE131.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 121 Stack Based Buffer Overflow
 * BadSource:  Allocate memory without using sizeof(int)
 * GoodSource: Allocate memory using sizeof(int)
 * Sink: loop
 *    BadSink : Copy array to data using a loop
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"


#ifndef OMITGOOD

/* goodG2B uses the GoodSource with the BadSink */
static void goodG2B()
{
    int * data;
    data = NULL;
    fake = NULL;
    /* FIX: Allocate memory using sizeof(int) */
    data = (int *)ALLOCA(10*sizeof(int));
    fake = (int *)ALLOCA(10);
    {
        int source[10] = {0};
        size_t i;
        /* POTENTIAL FLAW: Possible buffer overflow if data was not allocated correctly in the source */
        for (i = 0; i < 10; i++)
        {
            data[i] = source[i];
        }
        printIntLine(data[0]);
    }
}

void CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_good()
{
    goodG2B();
}

#endif /* OMITGOOD */

/* Below is the main(). It is only used when building this testcase on
 * its own for testing or for building a binary to use in testing binary
 * analysis tools. It is not used when compiling all the testcases as one
 * application, which is how source code analysis tools are tested.
 */

#ifdef INCLUDEMAIN

int main(int argc, char * argv[])
{
    /* seed randomness */
    srand( (unsigned)time(NULL) );
#ifndef OMITGOOD
    printLine("Calling good()...");
    CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_good();
    printLine("Finished good()");
#endif /* OMITGOOD */
    return 0;
}

#endif
"""

In [43]:
bad_fake = """/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01.c
Label Definition File: CWE121_Stack_Based_Buffer_Overflow__CWE131.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 121 Stack Based Buffer Overflow
 * BadSource:  Allocate memory without using sizeof(int)
 * GoodSource: Allocate memory using sizeof(int)
 * Sink: loop
 *    BadSink : Copy array to data using a loop
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"

#ifndef OMITBAD

void CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_bad()
{
    int * data;
    data = NULL;
    fake = NULL;
    /* FLAW: Allocate memory without using sizeof(int) */
    data = (int *)ALLOCA(10);
    fake = (int *)ALLOCA(10);
    {
        int source[10] = {0};
        size_t i;
        /* POTENTIAL FLAW: Possible buffer overflow if data was not allocated correctly in the source */
        for (i = 0; i < 10; i++)
        {
            data[i] = source[i];
        }
        printIntLine(data[0]);
    }
}

#endif /* OMITBAD */


/* Below is the main(). It is only used when building this testcase on
 * its own for testing or for building a binary to use in testing binary
 * analysis tools. It is not used when compiling all the testcases as one
 * application, which is how source code analysis tools are tested.
 */

#ifdef INCLUDEMAIN

int main(int argc, char * argv[])
{
    /* seed randomness */
    srand( (unsigned)time(NULL) );
#ifndef OMITBAD
    printLine("Calling bad()...");
    CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_bad();
    printLine("Finished bad()");
#endif /* OMITBAD */
    return 0;
}

#endif
"""

In [44]:
new_ilp_good = ilp_data[ilp_data['testcase_ID'] == 62804].copy()
new_ilp_good['code'] = good_fake
new_ilp_good['testcase_ID'] = 8554013
new_ilp_good['flaw_loc'] += 2

new_ilp_bad = ilp_data[ilp_data['testcase_ID'] == -62804].copy()
new_ilp_bad['code'] = bad_fake
new_ilp_bad['testcase_ID'] = -8554013
new_ilp_bad['flaw_loc'] += 2

In [45]:
ilp_data_force = pd.concat([ilp_data, new_ilp_good, new_ilp_bad])

In [46]:
good_fake = """
/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01.c
Label Definition File: CWE121_Stack_Based_Buffer_Overflow__CWE131.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 121 Stack Based Buffer Overflow
 * BadSource:  Allocate memory without using sizeof(int)
 * GoodSource: Allocate memory using sizeof(int)
 * Sink: loop
 *    BadSink : Copy array to data using a loop
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"


#ifndef OMITGOOD

/* goodG2B uses the GoodSource with the BadSink */
static void goodG2B()
{
    int * data;
    data = NULL;
    fake = NULL;
    /* FIX: Allocate memory using sizeof(int) */
    data = (int *)ALLOCA(10*sizeof(int));
    fake = (int *)ALLOCA(10*sizeof(int));
    {
        int source[10] = {0};
        size_t i;
        /* POTENTIAL FLAW: Possible buffer overflow if data was not allocated correctly in the source */
        for (i = 0; i < 10; i++)
        {
            data[i] = source[i];
        }
        printIntLine(data[0]);
    }
}

void CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_good()
{
    goodG2B();
}

#endif /* OMITGOOD */

/* Below is the main(). It is only used when building this testcase on
 * its own for testing or for building a binary to use in testing binary
 * analysis tools. It is not used when compiling all the testcases as one
 * application, which is how source code analysis tools are tested.
 */

#ifdef INCLUDEMAIN

int main(int argc, char * argv[])
{
    /* seed randomness */
    srand( (unsigned)time(NULL) );
#ifndef OMITGOOD
    printLine("Calling good()...");
    CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_good();
    printLine("Finished good()");
#endif /* OMITGOOD */
    return 0;
}

#endif
"""

In [47]:
bad_fake = """/* TEMPLATE GENERATED TESTCASE FILE
Filename: CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01.c
Label Definition File: CWE121_Stack_Based_Buffer_Overflow__CWE131.label.xml
Template File: sources-sink-01.tmpl.c
*/
/*
 * @description
 * CWE: 121 Stack Based Buffer Overflow
 * BadSource:  Allocate memory without using sizeof(int)
 * GoodSource: Allocate memory using sizeof(int)
 * Sink: loop
 *    BadSink : Copy array to data using a loop
 * Flow Variant: 01 Baseline
 *
 * */

#include "std_testcase.h"

#ifndef OMITBAD

void CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_bad()
{
    int * data;
    data = NULL;
    fake = NULL;
    /* FLAW: Allocate memory without using sizeof(int) */
    data = (int *)ALLOCA(10);
    fake = (int *)ALLOCA(10*sizeof(int));
    {
        int source[10] = {0};
        size_t i;
        /* POTENTIAL FLAW: Possible buffer overflow if data was not allocated correctly in the source */
        for (i = 0; i < 10; i++)
        {
            data[i] = source[i];
        }
        printIntLine(data[0]);
    }
}

#endif /* OMITBAD */


/* Below is the main(). It is only used when building this testcase on
 * its own for testing or for building a binary to use in testing binary
 * analysis tools. It is not used when compiling all the testcases as one
 * application, which is how source code analysis tools are tested.
 */

#ifdef INCLUDEMAIN

int main(int argc, char * argv[])
{
    /* seed randomness */
    srand( (unsigned)time(NULL) );
#ifndef OMITBAD
    printLine("Calling bad()...");
    CWE121_Stack_Based_Buffer_Overflow__CWE131_loop_01_bad();
    printLine("Finished bad()");
#endif /* OMITBAD */
    return 0;
}

#endif
"""

In [48]:
new_ilp_good = ilp_data[ilp_data['testcase_ID'] == 62804].copy()
new_ilp_good['code'] = good_fake
new_ilp_good['testcase_ID'] = 8554014
new_ilp_good['flaw_loc'] += 2

new_ilp_bad = ilp_data[ilp_data['testcase_ID'] == -62804].copy()
new_ilp_bad['code'] = bad_fake
new_ilp_bad['testcase_ID'] = -8554014
new_ilp_bad['flaw_loc'] += 2

In [49]:
ilp_data_force = pd.concat([ilp_data_force, new_ilp_good, new_ilp_bad])

In [50]:
ilp_data_force.to_csv("../data/ilp_dataset_force.csv.gz")

## Generating Prolog Representations

Now that we have a set of examples, we want to generate a Prolog representation. To do this we use Joern to derive a code property graph from each source file. Then, using our `../code/ILP-joern-cfg-to-prolog.scala` script, we convert a subset of this graph into a set of prolog facts. 

In [51]:
import os
import subprocess
import tempfile

In [52]:
testcase_IDs = []
flaws = []
bugs = []
code_lengths = []
trees = []

In [53]:
def generate_prolog(testcase):
    tmp_dir = tempfile.TemporaryDirectory()

    for file in testcase.itertuples():
        short_filename = file.filename.split("/")[-1]
        with open(tmp_dir.name + "/" + short_filename, 'w') as f:
            f.write(file.code)

    subprocess.check_call(["/joern/joern-parse", "--out", tmp_dir.name + "/cpg.bin.zip", tmp_dir.name])

    tree = subprocess.check_output(
        "cd /joern && /joern/joern-query --cpg "+tmp_dir.name + "/cpg.bin.zip -f /project/code/ILP-joern-cfg-to-prolog.scala",
        shell=True,
        universal_newlines=True,
    )

    testcase_IDs.append(file.testcase_ID)
    flaws.append( file.flaw)
    bugs.append(file.bug)
    code_lengths.append(file.code_length)
    trees.append(tree)

    tmp_dir.cleanup()

In [54]:
ilp_data_force.groupby('testcase_ID').apply(generate_prolog)

""


In [55]:
prolog = pd.DataFrame({
    'testcase_ID': testcase_IDs,
    'flaw': flaws,
    'bug': bugs,
    'code_length': code_lengths,
    'tree': trees,
})
prolog = prolog[1:]  # when we run apply it duplicates the first group
prolog

,testcase_ID,flaw,bug,code_length,tree
1,-8554014,CWE-121,True,1618,% START: Generated Prolog\n% NODE PROPERTIES \...
2,-8554013,CWE-121,True,1618,% START: Generated Prolog\n% NODE PROPERTIES \...
3,-232086,CWE-122,True,1625,% START: Generated Prolog\n% NODE PROPERTIES \...
4,-232012,CWE-122,True,1619,% START: Generated Prolog\n% NODE PROPERTIES \...
5,-62917,CWE-121,True,1622,% START: Generated Prolog\n% NODE PROPERTIES \...
6,-62916,CWE-121,True,1649,% START: Generated Prolog\n% NODE PROPERTIES \...
7,-62915,CWE-121,True,1638,% START: Generated Prolog\n% NODE PROPERTIES \...
8,-62913,CWE-121,True,1663,% START: Generated Prolog\n% NODE PROPERTIES \...
9,-62909,CWE-121,True,1655,% START: Generated Prolog\n% NODE PROPERTIES \...
10,-62902,CWE-121,True,1636,% START: Generated Prolog\n% NODE PROPERTIES \...


In [56]:
print(prolog.tree.iloc[0])

% START: Generated Prolog
% NODE PROPERTIES 
assignment(id_24_l_).
sizeOf(id_28_l_).
compMemberAccess(id_49_l_).
alloc(id_72_l_).
compMemberAccess(id_112_l_36).
compMemberAccess(id_117_l_34).
compMemberAccess(id_120_l_).
compMemberAccess(id_121_l_34).
assignment(id_122_l_34).
assignment(id_133_l_32).
assignment(id_137_l_29).
sizeOf(id_141_l_).
sizeOf(id_142_l_27).
alloc(id_146_l_27).
assignment(id_150_l_27).
alloc(id_152_l_).
alloc(id_153_l_26).
assignment(id_158_l_26).
assignment(id_161_l_24).
assignment(id_164_l_).
assignment(id_165_l_23).
% METHOD 
pointer(id_11_l_52).
voidPointer(id_99_l_53).
pointer(id_101_l_52).
pointer(id_111_l_36).
sizeOfInt(id_115_l_34).
sizeOfInt(id_116_l_34).
array10(id_116_l_34).
sizeOfInt(id_118_l_34).
pointer(id_119_l_34).
voidPointer(id_123_l_33).
sizeOfInt(id_124_l_32).
sizeOfInt(id_128_l_32).
sizeOfInt(id_132_l_32).
sizeOfInt(id_135_l_).
sizeOfInt(id_136_l_29).
array10(id_136_l_29).
sizeOfInt(id_138_l_).
array10(id_138_l_).
voidPointer(id_139_l_30).
po

In [57]:
import re

In [58]:
def fix_single_rules(testcase):
    find_node_ids = re.compile('\((\w+)\)\.')
    replacement_node_ids = '({bug}_{testcase_id}_\\1).'.format(
        bug='bad' if testcase.bug else 'good',
        testcase_id=abs(testcase.testcase_ID),
    )
    return find_node_ids.sub(replacement_node_ids, testcase['tree'])

def fix_tree_rules(testcase):
    find_node_ids = re.compile('\((\w+), (\w+)\)\.')
    replacement_node_ids = '({bug}_{testcase_id}_\\1, {bug}_{testcase_id}_\\2).'.format(
        bug='bad' if testcase.bug else 'good',
        testcase_id=abs(testcase.testcase_ID),
    )
    return find_node_ids.sub(replacement_node_ids, testcase['tree'])

def fix_code_rules(testcase):
    find_node_ids = re.compile('\((\w+), "(.*)"\)\.')
    replacement_node_ids = '({bug}_{testcase_id}_\\1, "\\2").'.format(
        bug='bad' if testcase.bug else 'good',
        testcase_id=abs(testcase.testcase_ID),
    )
    return find_node_ids.sub(replacement_node_ids, testcase['tree'])

In [59]:
prolog['tree'] = prolog.apply(fix_single_rules, axis='columns')
prolog['tree'] = prolog.apply(fix_tree_rules, axis='columns')
prolog['tree'] = prolog.apply(fix_code_rules, axis='columns')

In [60]:
prolog

,testcase_ID,flaw,bug,code_length,tree
1,-8554014,CWE-121,True,1618,% START: Generated Prolog\n% NODE PROPERTIES \...
2,-8554013,CWE-121,True,1618,% START: Generated Prolog\n% NODE PROPERTIES \...
3,-232086,CWE-122,True,1625,% START: Generated Prolog\n% NODE PROPERTIES \...
4,-232012,CWE-122,True,1619,% START: Generated Prolog\n% NODE PROPERTIES \...
5,-62917,CWE-121,True,1622,% START: Generated Prolog\n% NODE PROPERTIES \...
6,-62916,CWE-121,True,1649,% START: Generated Prolog\n% NODE PROPERTIES \...
7,-62915,CWE-121,True,1638,% START: Generated Prolog\n% NODE PROPERTIES \...
8,-62913,CWE-121,True,1663,% START: Generated Prolog\n% NODE PROPERTIES \...
9,-62909,CWE-121,True,1655,% START: Generated Prolog\n% NODE PROPERTIES \...
10,-62902,CWE-121,True,1636,% START: Generated Prolog\n% NODE PROPERTIES \...


In [61]:
def extract_source_map(prolog_src):
    output_lines = []
    
    in_source_code_section = False
    
    for line in prolog_src.split("\n"):
        if line == "% CODE":
            in_source_code_section = True
            
        if line == "% AST":
            in_source_code_section = False
        
        if in_source_code_section:
            output_lines.append(line)
            
    return '\n'.join(output_lines)
            
    
def remove_source_map(prolog_src):
    output_lines = []
    
    in_source_code_section = False
    
    for line in prolog_src.split("\n"):
        if line == "% CODE":
            in_source_code_section = True
            
        if line == "% AST":
            in_source_code_section = False
        
        if not in_source_code_section:
            output_lines.append(line)
            
    return '\n'.join(output_lines)

In [62]:
prolog['source_map'] = prolog['tree'].apply(extract_source_map)
prolog['tree'] = prolog['tree'].apply(remove_source_map)

In [63]:
prolog.to_csv("../data/ilp_prolog_data_force.csv.gz")